In [1]:
#!c1.8
%pip install -U pip icecream seaborn scikit-learn

In [40]:
#!c1.8
import pandas as pd
import numpy as np

import os
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from icecream import ic
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

random_state = 42

In [42]:
#!c1.8

def prepare_df(df):
    
    # Преобразуем время создания к типу datetime
    df.created = pd.to_datetime(df.created)
    # Выделим дату
    df['date'] = df.created.dt.date
    # Выделим номер месяца
    df['month'] = df.created.dt.month
    # Выделим номер дня недели
    df['weekdayid'] = pd.to_datetime(df.date).dt.weekday
    # Убираем пустые даты
    df = df[df.created!='1970-01-01 03:00:00']
    df.created = pd.to_datetime(df.created)
    df.fillna('', inplace=True)
    # Добавляем в подкатегирии категирии, в города области, в версии ос наименования ос во избежание неверной идентификации
    df.subgamecategory = df.gamecategory + '/' + df.subgamecategory
    df.city = df.oblast + '/' + df.city
    df.osv = df.os + '/' + df.osv
    df.os = df.os.str.lower()
    
    dfr = pd.DataFrame()

    if 'Segment' in df.columns:
        dfr['segment'] = df['Segment']

    # Разобьем время на 5-минутные интервалы (условно-минималная сессия без прерываний видео)
    # если в рамках одной сессии показано несколько видео, то фиксируем нескольких пользователей
    dfr['5min'] = (df.created.sub(pd.to_datetime('2021-01-01')).dt.total_seconds() // 300).astype(int)
    dfr['weekdayid'] = df.weekdayid
    dfr['month'] = df.month
    gamecategory_list = df['gamecategory'].unique()
    gamecategory_to_gcid = dict(zip(gamecategory_list, np.arange(len(gamecategory_list))))
    dfr['gcid'] = df['gamecategory'].map(gamecategory_to_gcid)
    subgamecategory_list = df['subgamecategory'].sort_values().unique()
    subgamecategory_to_sgcid = dict(zip(subgamecategory_list, np.arange(len(subgamecategory_list))))
    dfr['sgcid'] = df['subgamecategory'].map(subgamecategory_to_sgcid)
    bundle_list = df['bundle'].unique()
    bundle_to_bid = dict(zip(bundle_list, np.arange(len(bundle_list))))
    dfr['bid'] = df['bundle'].map(bundle_to_bid)
    shift_list = df['shift'].unique()
    shift_to_tsid = dict(zip(shift_list, np.arange(len(shift_list))))
    dfr['tsid'] = df['shift'].map(shift_to_tsid)
    oblast_list = df['oblast'].unique()
    oblast_to_oid = dict(zip(oblast_list, np.arange(len(oblast_list))))
    dfr['oid'] = df['oblast'].map(oblast_to_oid)
    city_list = df['city'].unique()
    city_to_cid = dict(zip(city_list, np.arange(len(city_list))))
    dfr['cid'] = df['city'].map(city_to_cid)
    os_list = df['os'].unique()
    os_to_osid = dict(zip(os_list, np.arange(len(os_list))))
    dfr['osid'] = df['os'].map(os_to_osid)
    osv_list = df['osv'].unique()
    osv_to_osvid = dict(zip(osv_list, np.arange(len(osv_list))))
    dfr['osvid'] = df['osv'].map(osv_to_osvid)
    return dfr


dates = [
    '2021-07-05',
    '2021-07-06',
    '2021-07-07',
    '2021-07-08',
    '2021-07-09',
    '2021-07-10',
    '2021-07-11',
    '2021-08-02',
    '2021-08-03',
    '2021-08-04',
    '2021-08-05',
    '2021-08-06',
    '2021-08-07',
    '2021-08-08',
    '2021-09-15',
    '2021-09-16',
    '2021-09-17',
    '2021-09-18',
    '2021-09-19',
    '2021-09-20',
]

for d in dates:
    ic()
    df_train = pd.read_csv(f"data/train_{d}.csv.zip")
    df_prep = prepare_df(df_train)

    X = df_prep.iloc[:,1:]
    y = df_prep.iloc[:,0]
    model = RandomForestClassifier(n_estimators=25, max_depth=15, random_state=random_state, n_jobs=8)
    model.fit(X, y)

    df_test = pd.read_csv(f"data/test_{d}.csv.zip", index_col=0)
    
    X_test = prepare_df(df_test)
    
    y_pred=model.predict(X_test)
    y_score=model.predict_proba(X_test)
    pd.DataFrame(y_score, columns=model.classes_, index = df_test.index).to_csv(f"results_{d}.csv", index=True, compression='zip')
    os.rename(f"results_{d}.csv", f"data/results_{d}.zip")

In [48]:
#!c1.8
with open('rfc.pkl', 'wb') as f:
    pickle.dump(model, f)

In [26]:
#!c1.8
df_test.index

In [30]:
#!c1.8
p = pd.DataFrame(y_score)

In [36]:
#!c1.8
y_score

In [38]:
#!c1.8
model.classes_

In [25]:
#!c1.8
pd.DataFrame(y_score, columns=[1,2,3,4,5], index = df_test.index)

In [17]:
#!c1.8
df_train

In [3]:
#!c1.8
ic()
df = pd.read_csv('data/train_2021-09-15.csv.zip')
ic()
# Преобразуем время создания к типу datetime
df.created = pd.to_datetime(df.created)
# Выделим дату
df['date'] = df.created.dt.date
# Выделим номер месяца
df['month'] = df.created.dt.month
# Выделим номер дня недели
df['weekdayid'] = pd.to_datetime(df.date).dt.weekday
ic()

In [4]:
#!c1.8
ic()

# Убираем путсые даты
df = df[df.created!='1970-01-01 03:00:00']
df.created = pd.to_datetime(df.created)
df.fillna('', inplace=True)

# Добавляем в подкатегирии категирии, в города области, в версии ос наименования ос во избежание неверной идентификации
df.subgamecategory = df.gamecategory + '/' + df.subgamecategory
df.city = df.oblast + '/' + df.city
df.osv = df.os + '/' + df.osv
df.os = df.os.str.lower()
ic()

In [5]:
#!c1.8
ic()
display(df.describe(include='all'))
ic()

In [6]:
#!c1.8
ic()
display(df)
ic()

In [7]:
#!c1.8
ic()
dfr = pd.DataFrame(columns = ['segment', '5min', 'weekdayid', 'month', 'gcid', 'sgcid', 'bid', 'tsid', 'oid', 'cid', 'osid', 'osvid'])

# Сформируем для каждой записи порядковый номер на временнй шкале
# dfr['dtid'] = df.index.map(dict(zip(df.sort_values('created').index, np.arange(len(df)))))
# ic()

# Разобьем время на 5-минутные интервалы (условно-минималная сессия без прерываний видео)
# если в рамках одной сессии показано несколько видео, то фиксируем нескольких пользователей
dfr['5min'] = (df.created.sub(pd.to_datetime('2021-01-01')).dt.total_seconds() // 300).astype(int)
ic()

dfr['weekdayid'] = df.weekdayid
ic()

dfr['month'] = df.month
ic()

dfr['segment'] = df['Segment']
ic()
gamecategory_list = df['gamecategory'].unique()
gamecategory_to_gcid = dict(zip(gamecategory_list, np.arange(len(gamecategory_list))))
dfr['gcid'] = df['gamecategory'].map(gamecategory_to_gcid)
ic()
subgamecategory_list = df['subgamecategory'].sort_values().unique()
subgamecategory_to_sgcid = dict(zip(subgamecategory_list, np.arange(len(subgamecategory_list))))
dfr['sgcid'] = df['subgamecategory'].map(subgamecategory_to_sgcid)
ic()
bundle_list = df['bundle'].unique()
bundle_to_bid = dict(zip(bundle_list, np.arange(len(bundle_list))))
dfr['bid'] = df['bundle'].map(bundle_to_bid)
ic()
shift_list = df['shift'].unique()
shift_to_tsid = dict(zip(shift_list, np.arange(len(shift_list))))
dfr['tsid'] = df['shift'].map(shift_to_tsid)
ic()
oblast_list = df['oblast'].unique()
oblast_to_oid = dict(zip(oblast_list, np.arange(len(oblast_list))))
dfr['oid'] = df['oblast'].map(oblast_to_oid)
ic()
city_list = df['city'].unique()
city_to_cid = dict(zip(city_list, np.arange(len(city_list))))
dfr['cid'] = df['city'].map(city_to_cid)
ic()
os_list = df['os'].unique()
os_to_osid = dict(zip(os_list, np.arange(len(os_list))))
dfr['osid'] = df['os'].map(os_to_osid)
ic()
osv_list = df['osv'].unique()
osv_to_osvid = dict(zip(osv_list, np.arange(len(osv_list))))
dfr['osvid'] = df['osv'].map(osv_to_osvid)
ic()

In [8]:
#!c1.8
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score
random_state = 42

In [9]:
#!c1.8
ic()
X = dfr.iloc[:,1:]
y = dfr.iloc[:,0]
ic()

In [10]:
#!c1.8
pd.set_option('display.max_rows', 10)
display(X)

In [11]:
#!c1.8
ic()
pd.set_option('display.max_rows', 10)
display(y)
ic()

In [12]:
#!c1.8
ic()
y = label_binarize(y, classes=[1, 2, 3, 4, 5])
n_classes = y.shape[1]
display(y.shape)
ic()

In [13]:
#!c1.8
ic()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=random_state)
ic()

In [14]:
#!c1.8
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
#!c1.8
